In [5]:
import cv2
import dlib   # 스노우 얼굴인식 카메라. 얼굴 영역 탐지, 랜드마크 탐지 이용
from imutils import face_utils, resize  
import numpy as np

In [9]:
orange_img = cv2.imread('../orange.jpg')
orange_img = cv2.resize(orange_img, dsize=(512, 512))

detector = dlib.get_frontal_face_detector()   # 얼굴 영역 탐지
predictor = dlib.shape_predictor('../shape_predictor_68_face_landmarks.dat')  # 68점 랜드마크 탐지

cap = cv2.VideoCapture(0)  # 0이면 웹캠. 파일명이면 비디오를 재생
# cap = cv2.VideoCapture('01.mp4')  

while cap.isOpened():
    ret, img = cap.read()

    if not ret:  # 프레임이 더 없으면 반복문 나가기
        break

    faces = detector(img)   # 이미지에서 영역을 인식하고 좌표를 반환

    result = orange_img.copy()

    if len(faces) > 0:    
        face = faces[0]   # 얼굴 1개만 인식할테니까 
        
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        face_img = img[y1:y2, x1:x2].copy()

        # 랜드마크 68개의 점의 좌표를 구한다.
        shape = predictor(img, face)
        shape = face_utils.shape_to_np(shape)  # dlib을 np로 바꿔줌

        # for p in shape:
        #     cv2.circle(face_img, center=(p[0] - x1, p[1] - y1), radius=2, color=255, thickness=-1)

        # eyes   # 번호 인덱스가 고정되어있음. 
        le_x1 = shape[36, 0]
        le_y1 = shape[37, 1]
        le_x2 = shape[39, 0]
        le_y2 = shape[41, 1]
        le_margin = int((le_x2 - le_x1) * 0.18)   # 타이트하게 자르지 않도록

        re_x1 = shape[42, 0]   # 오른쪽 눈
        re_y1 = shape[43, 1]
        re_x2 = shape[45, 0]
        re_y2 = shape[47, 1]
        re_margin = int((re_x2 - re_x1) * 0.18)

        # crop
        left_eye_img = img[le_y1-le_margin:le_y2+le_margin, le_x1-le_margin:le_x2+le_margin].copy()
        right_eye_img = img[re_y1-re_margin:re_y2+re_margin, re_x1-re_margin:re_x2+re_margin].copy()

        # 가로 100사이즈로 resizing
        left_eye_img = resize(left_eye_img, width=100)
        right_eye_img = resize(right_eye_img, width=100)

        # 포아송블랜딩. 합성
        # 왼쪽 눈과 오른쪽 눈을 result(오렌지)에 합성
        result = cv2.seamlessClone(
            left_eye_img,
            result,
            np.full(left_eye_img.shape[:2], 255, left_eye_img.dtype),
            (100, 200),
            cv2.MIXED_CLONE   # 알아서 합성
        )

        result = cv2.seamlessClone(
            right_eye_img,
            result,
            np.full(right_eye_img.shape[:2], 255, right_eye_img.dtype),
            (250, 200),
            cv2.MIXED_CLONE
        )

        # mouth
        mouth_x1 = shape[48, 0]
        mouth_y1 = shape[50, 1]
        mouth_x2 = shape[54, 0]
        mouth_y2 = shape[57, 1]
        mouth_margin = int((mouth_x2 - mouth_x1) * 0.1)

        mouth_img = img[mouth_y1-mouth_margin:mouth_y2+mouth_margin, mouth_x1-mouth_margin:mouth_x2+mouth_margin].copy()

        mouth_img = resize(mouth_img, width=250)

        result = cv2.seamlessClone(
            mouth_img,
            result,
            np.full(mouth_img.shape[:2], 255, mouth_img.dtype),
            (180, 320),
            cv2.MIXED_CLONE
        )

        # cv2.imshow('left', left_eye_img)
        # cv2.imshow('right', right_eye_img)
        # cv2.imshow('mouth', mouth_img)
        # cv2.imshow('face', face_img)

        cv2.imshow('result', result)

    # cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

KeyboardInterrupt: 